# Prepare your data

This tutorial guides you from an FCS file to the creation of a `Scyan` object.

## 1. Creation of your `AnnData` object (cytometry data)

The first step is to create an `AnnData` object containing your cytometry data. Consider reading the [anndata documentation](https://anndata.readthedocs.io/en/latest/) if you have never heard about `anndata` before.

NB: Make sure you only take the population of interest. E.g., if you are interested into immune cells, consider providing only the live cells that are CD45+.

### Loading a `FCS` file

You probably have `.fcs` files that you want to load. For this, you can use [`scyan.read_fcs`](../../api/io/#scyan.read_fcs). Make sure you have already [installed scyan](../../getting_started).

NB: if you have another data format, such as `csv`, it also works: you just have to make your own [anndata](https://anndata.readthedocs.io/en/latest/) object.

In [1]:
import scyan

Global seed set to 0


In [2]:
adata = scyan.read_fcs("<path-to-fcs>.fcs")

On this example, we have $N = 52 981$ cells and $M = 38$ markers (see below).
Make sure your marker names looks good, or consider reading [`scyan.read_fcs`](../../api/io/#scyan.read_fcs) for more advanced usage.

If you have multiple `FCS`, consider [concatenating your data](https://anndata.readthedocs.io/en/latest/generated/anndata.AnnData.concatenate.html#anndata.AnnData.concatenate).

In [3]:
print(adata)
print(f"\nThe markers names are: {', '.join(adata.var_names)}")

AnnData object with n_obs × n_vars = 52981 × 38
    obs: 'FSC-A', ..., 'Time'

The markers names are: CD8, CD4, ...


### Preprocess your data

Choose either the `asinh` or `logicle` transformation below, and scale your data.

In [4]:
is_cytof = True

if is_cytof: # we recommend asinh for CyTOF data
    scyan.tools.asinh_transform(adata)
else: # we recommend auto_logicle for flow or spectral flow
    scyan.tools.auto_logicle_transform(adata)

scyan.tools.scale(adata) # To standardise your data

## 2. (Optional) Compute a UMAP
You can compute the UMAP coordinates using [`scyan.tools.umap`](../../api/representation/#scyan.tools.umap). The API will guide you for the usage of this tool (especially, you can choose to compute the UMAP on a specific set of markers, or choose a subset of cells on which computing the UMAP for acceleration).

Note that it only computes the coordinates, then you'll have to use [`scyan.plot.umap`](../../api/plots/#scyan.plot.umap) to display it.

In [ ]:
scyan.tools.umap(adata);

## 3. Creation of the knowledge table
The knowledge table, or marker-population table, contains well-known marker expressions per population. For instance, if you want `Scyan` to annotate CD4 T cells, you have to tell which markers CD4 T cells are supposed to express or not. Depending on your panel, you may have CD4+, CD8-, CD45+, CD3+, etc. Values inside the table can be:

- `-1` for negative expressions.
- `1` for positive expressions.
- Some float values such as `0` or `0.5` for mid and low expressions respectively (use it only when necessary).
- `NA` when you don't know or if it is not applicable (you can also let the field empty, it will be read as `NA`).

We recommend the `csv` format for this table. You can either directly create a `csv`, or use Excel and export the table as `csv`.

You can then import the `csv` to make a pandas `DataFrame`.

### Example

In [5]:
import pandas as pd

In [6]:
table = pd.read_csv("<path-to-csv>.csv", index_col=0)

In [7]:
table.head() # Display the first 5 rows of the table

,CD19,CD4,CD8,CD34,CD20,CD45,CD123,CD11c,CD7,CD16,CD38,CD3,HLA-DR,CD64
Populations,,,,,,,,,,,,,,
Basophils,-1,NaN,-1.0,-1,-1.0,NaN,1,-1,-1.0,-1.0,NaN,-1,-1.0,-1.0
CD4 T cells,-1,1.0,-1.0,-1,-1.0,NaN,-1,-1,NaN,-1.0,NaN,1,-1.0,-1.0
CD8 T cells,-1,-1.0,1.0,-1,-1.0,NaN,-1,-1,1.0,-1.0,NaN,1,-1.0,-1.0
CD16- NK cells,-1,NaN,NaN,-1,-1.0,NaN,-1,-1,1.0,-1.0,NaN,-1,-1.0,-1.0
CD16+ NK cells,-1,NaN,NaN,-1,NaN,NaN,-1,-1,1.0,1.0,NaN,-1,-1.0,-1.0


You can see our [advice](../../advanced/advice) when creating this table.

Also, ensure your column names correspond to marker names in `adata.var_names`.

## 4. Creation of the `Scyan` model

In [8]:
model = scyan.Scyan(adata, table)

INFO:scyan.model:Initialized Scyan model with N=52981 cells, P=29 populations and M=38 markers. No covariate provided.


Congratulations! You can now follow our tutorial on [model training and visualisation](../../tutorials/usage).

## 5. (Optional) Save your data for later use

You can use [scyan.data.add](../../api/datasets/#scyan.data.add) to save your data.

In [9]:
scyan.data.add("your-project-name", adata, table)

INFO:scyan.data.datasets:Creating new dataset folder at /.../your_project_name
INFO:scyan.data.datasets:Created file /.../your_project_name/default.h5ad
INFO:scyan.data.datasets:Created file /.../your_project_name/default.csv


You can now simply load it with [scyan.data.load](../../api/datasets/#scyan.data.load).

In [10]:
adata, table = scyan.data.load("your-project-name")